In [42]:
print(f"""
    Description
        baseline 코드 
        
    - date
        since 2021-12-04
    
    - version
        v1.0
            * notebook 생성
            * tutorial notebook 참고하여 기본 feature 생성
""")


    Description
        baseline 코드 
        
    - date
        since 2021-12-04
    
    - version
        v1.0
            * notebook 생성
            * tutorial notebook 참고하여 기본 feature 생성



# 0 data load

In [43]:
import numpy as np
import pandas as pd
from datetime import datetime

In [44]:
dpath = "C:/Users/minho/kaggle dataset/g-research-crypto-forecasting/input/"
raw = pd.read_csv(dpath + 'train.csv')
raw_asset_details = pd.read_csv(dpath + 'asset_details.csv')
raw_example_test = pd.read_csv(dpath + 'example_test.csv')

In [45]:
raw.head(10)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.580000,2399.5000,2357.1400,2374.590000,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.530000,8.5300,8.5300,8.530000,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.194000,14013.8000,13666.1100,13850.176000,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.659600,7.6596,7.6567,7.657600,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.920000,25.9200,25.8740,25.877000,121.087310,25.891363,-0.008264
5,1514764860,6,173.0,738.302500,746.0000,732.5100,738.507500,335.987856,738.839291,-0.004809
6,1514764860,9,167.0,225.330000,227.7800,222.9800,225.206667,411.896642,225.197944,-0.009791
7,1514764860,11,7.0,329.090000,329.8800,329.0900,329.460000,6.635710,329.454118,NaN
8,1514764920,2,53.0,2374.553333,2400.9000,2354.2000,2372.286667,24.050259,2371.434498,-0.004079
9,1514764920,0,7.0,8.530000,8.5300,8.5145,8.514500,71.390000,8.520215,-0.015875


In [46]:
raw_asset_details

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


## 1. bitcoin set

In [47]:
df_btc = raw[raw['Asset_ID'] == 1].set_index('timestamp')
df_btc

,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
timestamp,,,,,,,,,
1514764860,1,229.0,13835.194000,14013.800000,13666.11,13850.176000,31.550062,13827.062093,-0.014643
1514764920,1,235.0,13835.036000,14052.300000,13680.00,13828.102000,31.046432,13840.362591,-0.015037
1514764980,1,528.0,13823.900000,14000.400000,13601.00,13801.314000,55.061820,13806.068014,-0.010309
1514765040,1,435.0,13802.512000,13999.000000,13576.28,13768.040000,38.780529,13783.598101,-0.008999
1514765100,1,742.0,13766.000000,13955.900000,13554.44,13724.914000,108.501637,13735.586842,-0.008079
...,...,...,...,...,...,...,...,...,...
1632182160,1,1940.0,42983.780000,43001.850849,42878.26,42899.012857,56.850913,42935.489499,NaN
1632182220,1,2026.0,42904.197143,42932.000000,42840.16,42860.005714,80.993326,42879.576084,NaN
1632182280,1,1986.0,42859.385714,42887.500000,42797.20,42827.020000,65.677734,42844.090693,NaN


In [48]:
df_btc.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956282 entries, 1514764860 to 1632182400
Data columns (total 9 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   Asset_ID  1956282 non-null  int64  
 1   Count     1956282 non-null  float64
 2   Open      1956282 non-null  float64
 3   High      1956282 non-null  float64
 4   Low       1956282 non-null  float64
 5   Close     1956282 non-null  float64
 6   Volume    1956282 non-null  float64
 7   VWAP      1956282 non-null  float64
 8   Target    1955978 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 149.3 MB


    비트코인 테이블 내 코인의 시작일시와 끝 일시

In [53]:
nl = '\n'
start_btc = df_btc.index[0].astype('datetime64[s]')
end_btc = df_btc.index[-1].astype('datetime64[s]')
by_btc = df_btc.index[1] - df_btc.index[0]
length_btc = len(df_btc)

msg = f'''Bitcoin 
    start date : {start_btc}       
    end date : {end_btc}    
    by : {by_btc} seconds
    length : {length_btc}
    '''
print(msg)

Bitcoin 
    start date : 2018-01-01T00:01:00       
    end date : 2021-09-21T00:00:00    
    by : 60 seconds
    length : 1956282
    


    비트코인 시계열에서 index 결측치

In [9]:
(df_btc.index[1:]-df_btc.index[:-1]).value_counts().head()

60     1956136
120         78
180         12
240         11
420          9
Name: timestamp, dtype: int64

In [10]:
df_btc.isna().sum()

Asset_ID      0
Count         0
Open          0
High          0
Low           0
Close         0
Volume        0
VWAP          0
Target      304
dtype: int64

## 1-1 imputation
첫 번째 : reindex pad

    중간에 빈 시간대의 값은 직전값으로 보정해 주는 작업이다.

In [11]:
df_btc = df_btc.reindex(range(df_btc.index[0],
                              df_btc.index[-1] + 60, 60),
                        method = "pad")

In [13]:
(df_btc.index[1:]-df_btc.index[:-1]).value_counts().head()

60    1956959
Name: timestamp, dtype: int64

In [39]:
# 총 데이터는 1956960 개임.
df_btc.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1956960 entries, 1514764860 to 1632182400
Data columns (total 9 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   Asset_ID  1956960 non-null  int64  
 1   Count     1956960 non-null  float64
 2   Open      1956960 non-null  float64
 3   High      1956960 non-null  float64
 4   Low       1956960 non-null  float64
 5   Close     1956960 non-null  float64
 6   Volume    1956960 non-null  float64
 7   VWAP      1956960 non-null  float64
 8   Target    1956023 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 134.4 MB


In [82]:
df_btc

,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
timestamp,,,,,,,,,
1514764860,1,229.0,13835.194000,14013.800000,13666.11,13850.176000,31.550062,13827.062093,-0.014643
1514764920,1,235.0,13835.036000,14052.300000,13680.00,13828.102000,31.046432,13840.362591,-0.015037
1514764980,1,528.0,13823.900000,14000.400000,13601.00,13801.314000,55.061820,13806.068014,-0.010309
1514765040,1,435.0,13802.512000,13999.000000,13576.28,13768.040000,38.780529,13783.598101,-0.008999
1514765100,1,742.0,13766.000000,13955.900000,13554.44,13724.914000,108.501637,13735.586842,-0.008079
...,...,...,...,...,...,...,...,...,...
1632182160,1,1940.0,42983.780000,43001.850849,42878.26,42899.012857,56.850913,42935.489499,NaN
1632182220,1,2026.0,42904.197143,42932.000000,42840.16,42860.005714,80.993326,42879.576084,NaN
1632182280,1,1986.0,42859.385714,42887.500000,42797.20,42827.020000,65.677734,42844.090693,NaN


* 궁금한 점 검증
    1. 거래량이 폭증한 시간에 -> VWAP도 같이 급등한다?
    
       거래량 폭증 계산
           (t-1)시점에서의 거래량보다 t시점에서 거래량 %
           
        

* 가설
    1. 거래량이 폭증한 시간에 그래

In [90]:
df_btc['diff1_Volume'] = df_btc['Volume'].diff()

In [91]:
df_btc

,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,diff1_Volume
timestamp,,,,,,,,,,
1514764860,1,229.0,13835.194000,14013.800000,13666.11,13850.176000,31.550062,13827.062093,-0.014643,NaN
1514764920,1,235.0,13835.036000,14052.300000,13680.00,13828.102000,31.046432,13840.362591,-0.015037,-0.503629
1514764980,1,528.0,13823.900000,14000.400000,13601.00,13801.314000,55.061820,13806.068014,-0.010309,24.015388
1514765040,1,435.0,13802.512000,13999.000000,13576.28,13768.040000,38.780529,13783.598101,-0.008999,-16.281291
1514765100,1,742.0,13766.000000,13955.900000,13554.44,13724.914000,108.501637,13735.586842,-0.008079,69.721107
...,...,...,...,...,...,...,...,...,...,...
1632182160,1,1940.0,42983.780000,43001.850849,42878.26,42899.012857,56.850913,42935.489499,NaN,9.149634
1632182220,1,2026.0,42904.197143,42932.000000,42840.16,42860.005714,80.993326,42879.576084,NaN,24.142412
1632182280,1,1986.0,42859.385714,42887.500000,42797.20,42827.020000,65.677734,42844.090693,NaN,-15.315592
